In [1]:
from src.data_preprocessor import DataProcessor
from data_configs.configs import *
from models.neural_networks import *
from src.cross_validation import CrossValidation
import numpy as np

config = machine_config
data_processor = DataProcessor(config=config)
cross_validator = CrossValidation(config=config)

In [2]:
raw_data = data_processor.load_data()
data_1 = raw_data.drop(columns=['vendor_name','model_name','ERP'])
data_2 = data_processor.impute_missing_values(data_1)
data_3 = data_processor.standardize_data(data_2,data_2,features=['MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN', 'CHMAX'])


c:\Users\Srikanta\Documents\Intro to Machine Learning\programming_assignment_1\src\data_preprocessor.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[column].fillna(data[column].mean(), inplace=True)


In [3]:
data_3

,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP
0,-0.302859,-0.673409,-0.494275,5.680569,1.658008,4.220899,198
1,-0.671717,1.323114,1.722913,0.167228,0.484346,0.528211,269
2,-0.671717,1.323114,1.722913,0.167228,0.484346,0.528211,220
3,-0.671717,1.323114,1.722913,0.167228,0.484346,0.528211,172
4,-0.671717,1.323114,0.358489,0.167228,0.484346,-0.087238,132
...,...,...,...,...,...,...,...
204,-0.306701,-0.481594,-0.323723,-0.620392,-0.542608,-0.394962,42
205,-0.406600,-0.481594,-0.323723,0.167228,-0.395900,-0.394962,46
206,-0.302859,-0.223779,-0.323723,-0.620392,-0.395900,-0.164169,52
207,1.061146,-0.607408,-0.323723,0.167228,-0.689316,-0.702686,67


In [4]:
data_train, data_val = cross_validator.random_partition(data_3, random_state=42)

In [12]:
import random

iterations = 15

param_space = {
    'lr': [0.01,0.001,0.0001],
    'epochs': np.linspace(1000, 20000, num=20).astype(int).tolist()
}

best_score = float('inf')
best_params_linear = {}

for _ in range(iterations):

    # Randomly select parameters
    params = {key: random.choice(value) for key, value in param_space.items()}
    scores = []

    for i, (train_set, _) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
    
        train_data = train_set.to_numpy()
        X_train = train_data[:,:-1]
        y_train = train_data[:,-1:]

        data_test = data_val.to_numpy()
        X_val = data_test[:,:-1]
        y_val = data_test[:,-1:]
        
        linear = LinearNetwork(config)

        _, val_losses = linear.linear_regression(X_train,y_train,X_val,y_val,epochs=params['epochs'],lr=params['lr'],patience=500)

        score = np.min(val_losses)
        scores.append(score)

        # Skip to the next parameter set if score > 0.2
        if score > 20000:
            print(f"Skipping params: {params} due to high score: {score}")
            break  # Exit the current for-loop
        
    avg_score = np.mean(scores)

    print(f"Tested params: {params}, Score: {avg_score}")
    
    if avg_score < best_score:
        best_score = avg_score
        best_params_linear = params
        

print(f"Best parameters: {best_params_linear}, Best score: {best_score}")


Tested params: {'lr': 0.01, 'epochs': 3000}, Score: 7970.490785854759
Tested params: {'lr': 0.01, 'epochs': 5000}, Score: 7970.475786088262
Tested params: {'lr': 0.01, 'epochs': 20000}, Score: 7970.450695079872
Tested params: {'lr': 0.001, 'epochs': 5000}, Score: 8017.537044520643
Tested params: {'lr': 0.0001, 'epochs': 20000}, Score: 8490.875881480595
Tested params: {'lr': 0.001, 'epochs': 1000}, Score: 9890.905015994285
Tested params: {'lr': 0.01, 'epochs': 18000}, Score: 7970.524197738162
Tested params: {'lr': 0.01, 'epochs': 17000}, Score: 7970.460200168816
Tested params: {'lr': 0.001, 'epochs': 12000}, Score: 7974.60848587786
Tested params: {'lr': 0.001, 'epochs': 14000}, Score: 7973.364663793899
Tested params: {'lr': 0.0001, 'epochs': 20000}, Score: 8490.870895067274
Tested params: {'lr': 0.01, 'epochs': 5000}, Score: 7970.492238172022
Tested params: {'lr': 0.001, 'epochs': 11000}, Score: 7975.562366765633
Tested params: {'lr': 0.001, 'epochs': 8000}, Score: 7981.998455895757
Tes

In [15]:
import random

iterations = 15

param_space = {
    'lr': [0.0001,0.00001,0.000001,0.0000001],
    'epochs': np.arange(2000, 20000, 2000).tolist(),
    'n_hidden': np.arange(2*X_val.shape[1], 100, 15)
}

best_score = float('inf')
best_params_ffn = {}

for _ in range(iterations):

    # Randomly select parameters
    params = {key: random.choice(value) for key, value in param_space.items()}
    scores = []

    for i, (train_set, _) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
           
        train_data = train_set.to_numpy()
        X_train = train_data[:,:-1]
        y_train = train_data[:,-1:]

        data_test = data_val.to_numpy()
        X_val = data_test[:,:-1]
        y_val = data_test[:,-1:]

        ffn = FeedForwardNetwork(config,n_input=X_train.shape[1],n_hidden_1=params['n_hidden'],n_hidden_2=params['n_hidden'],n_output=y_train.shape[1])

        _, val_losses, _ = ffn.train(X_train,y_train,X_val,y_val,epochs=params['epochs'],lr=params['lr'],patience=500)

        score = np.min(val_losses)
        scores.append(score)

        # Skip to the next parameter set if score > 0.2
        if score > 20000:
            print(f"Skipping params: {params} due to high score: {score}")
            break  # Exit the current for-loop

    avg_score = np.mean(scores)

    print(f"Tested params: {params}, Score: {avg_score}")
    
    if avg_score < best_score:
        best_score = avg_score
        best_params_ffn = params
        

print(f"Best parameters: {best_params_ffn}, Best score: {best_score}")


Skipping params: {'lr': 0.0001, 'epochs': 4000, 'n_hidden': 12} due to high score: 41193.35520669441
Tested params: {'lr': 0.0001, 'epochs': 4000, 'n_hidden': 12}, Score: 41193.35520669441
Skipping params: {'lr': 1e-06, 'epochs': 14000, 'n_hidden': 42} due to high score: 21477.184125792883
Tested params: {'lr': 1e-06, 'epochs': 14000, 'n_hidden': 42}, Score: 21477.184125792883
Skipping params: {'lr': 1e-07, 'epochs': 12000, 'n_hidden': 57} due to high score: 43522.2152594878
Tested params: {'lr': 1e-07, 'epochs': 12000, 'n_hidden': 57}, Score: 43522.2152594878
Skipping params: {'lr': 1e-05, 'epochs': 2000, 'n_hidden': 27} due to high score: 21881.45524677582
Tested params: {'lr': 1e-05, 'epochs': 2000, 'n_hidden': 27}, Score: 21881.45524677582
Skipping params: {'lr': 1e-05, 'epochs': 2000, 'n_hidden': 42} due to high score: 20141.025311628873
Tested params: {'lr': 1e-05, 'epochs': 2000, 'n_hidden': 42}, Score: 20141.025311628873
Skipping params: {'lr': 1e-07, 'epochs': 8000, 'n_hidden'

In [16]:
import random

iterations = 15

param_space = {
    'lr': [0.001,0.0001,0.00001,0.000001,0.0000001],
    'epochs': np.arange(5000, 20000, 2000).tolist(),
    'n_encoder': np.arange(2,X_val.shape[1]-1,1).tolist()
}

best_score = float('inf')
best_params_auto = {}

for _ in range(iterations):

    # Randomly select parameters
    params = {key: random.choice(value) for key, value in param_space.items()}
    scores = []

    for i, (train_set, _) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
    
        train_data = train_set.to_numpy()
        X_train = train_data[:,:-1]
        y_train = train_data[:,-1:]

        data_test = data_val.to_numpy()
        X_val = data_test[:,:-1]
        y_val = data_test[:,-1:]

        # Check if the shapes of X_train and X_val are not equal
        if X_train.shape[1] != X_val.shape[1]:
            # print(f"Shape mismatch between training and validation sets, skipping params: {params}")
            continue

        autoE = AutoEncoder(config,n_input=X_train.shape[1],n_encoder=params['n_encoder'])

        losses = autoE.train(X_train, max_epochs=params['epochs'], lr=params['lr'])

        score = np.min(losses)
        scores.append(score)

        # Skip to the next parameter set if score > 0.2
        if score > 0.2:
            print(f"Skipping params: {params} due to high score.")
            break  # Exit the current for-loop

    avg_score = np.mean(scores)

    print(f"Tested params: {params}, Score: {avg_score}")
    
    if avg_score < best_score:
        best_score = avg_score
        best_params_auto = params
        

print(f"Best parameters: {best_params_auto}, Best score: {best_score}")


Skipping params: {'lr': 1e-06, 'epochs': 7000, 'n_encoder': 2} due to high score.
Tested params: {'lr': 1e-06, 'epochs': 7000, 'n_encoder': 2}, Score: 0.7451574617985534
Tested params: {'lr': 0.001, 'epochs': 5000, 'n_encoder': 4}, Score: 0.07360011892582566
Skipping params: {'lr': 1e-07, 'epochs': 17000, 'n_encoder': 3} due to high score.
Tested params: {'lr': 1e-07, 'epochs': 17000, 'n_encoder': 3}, Score: 0.7465465252665268
Skipping params: {'lr': 0.0001, 'epochs': 9000, 'n_encoder': 4} due to high score.
Tested params: {'lr': 0.0001, 'epochs': 9000, 'n_encoder': 4}, Score: 0.2544451637208205
Skipping params: {'lr': 1e-07, 'epochs': 13000, 'n_encoder': 4} due to high score.
Tested params: {'lr': 1e-07, 'epochs': 13000, 'n_encoder': 4}, Score: 0.746848557376376
Skipping params: {'lr': 1e-05, 'epochs': 11000, 'n_encoder': 3} due to high score.
Tested params: {'lr': 1e-05, 'epochs': 11000, 'n_encoder': 3}, Score: 0.7284968514102522
Skipping params: {'lr': 1e-07, 'epochs': 13000, 'n_enc

In [17]:
autoE = AutoEncoder(config,n_input=X_train.shape[1],n_encoder=best_params_auto['n_encoder'])
losses = autoE.train(X_train, max_epochs=best_params_auto['epochs'], lr=best_params_auto['lr'])
losses[-1]

0.0668874747296308

In [20]:
import random

iterations = 15

param_space = {
    'lr': [0.001,0.0001,0.00001,0.000001,0.0000001],
    'epochs': np.arange(5000, 20000, 2000).tolist(),
    'n_hidden_2': np.arange(X_val.shape[1], 100, 15)
}

best_score = float('inf')
best_params_combined = {}

for _ in range(iterations):

    # Randomly select parameters
    params = {key: random.choice(value) for key, value in param_space.items()}
    scores = []
    print(f"Testing params: {params}")

    for i, (train_set, _) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):
    
        train_data = train_set.to_numpy()
        X_train = train_data[:,:-1]
        y_train = train_data[:,-1:]

        data_test = data_val.to_numpy()
        X_val = data_test[:,:-1]
        y_val = data_test[:,-1:]

        # Check if the shapes of X_train and X_val are not equal
        if X_train.shape[1] != X_val.shape[1]:
            # print(f"Shape mismatch between training and validation sets, skipping params: {params}")
            continue
        
        combined = CombinedModel(autoE,n_hidden_2=params['n_hidden_2'],n_output=y_val.shape[1])

        _, val_losses, _ = combined.train(X_train,y_train,X_val,y_val,epochs=params['epochs'], lr=params['lr'],patience=500)


        score = np.min(val_losses)
        scores.append(score)

        # Skip to the next parameter set if score > 0.2
        if score > 20000:
            print(f"Skipping params: {params} due to high score: {score}")
            break  # Exit the current for-loop

    avg_score = np.mean(scores)

    print(f"Tested params: {params}, Score: {avg_score}")
    
    if avg_score < best_score:
        best_score = avg_score
        best_params_combined = params
        

print(f"Best parameters: {best_params_combined}, Best score: {best_score}")


Testing params: {'lr': 0.0001, 'epochs': 9000, 'n_hidden_2': 21}
Skipping params: {'lr': 0.0001, 'epochs': 9000, 'n_hidden_2': 21} due to high score: 22703.182169619064
Tested params: {'lr': 0.0001, 'epochs': 9000, 'n_hidden_2': 21}, Score: 22703.182169619064
Testing params: {'lr': 1e-07, 'epochs': 17000, 'n_hidden_2': 6}
Skipping params: {'lr': 1e-07, 'epochs': 17000, 'n_hidden_2': 6} due to high score: 57173.475358766984
Tested params: {'lr': 1e-07, 'epochs': 17000, 'n_hidden_2': 6}, Score: 57173.475358766984
Testing params: {'lr': 1e-05, 'epochs': 19000, 'n_hidden_2': 51}
Skipping params: {'lr': 1e-05, 'epochs': 19000, 'n_hidden_2': 51} due to high score: 20909.784938357796
Tested params: {'lr': 1e-05, 'epochs': 19000, 'n_hidden_2': 51}, Score: 14448.922452916402
Testing params: {'lr': 0.001, 'epochs': 11000, 'n_hidden_2': 51}


c:\Users\Srikanta\Documents\Intro to Machine Learning\programming_assignment_1\.venv\Lib\site-packages\numpy\core\_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
c:\Users\Srikanta\Documents\Intro to Machine Learning\programming_assignment_1\models\neural_networks.py:459: RuntimeWarning: overflow encountered in square
  train_metric = np.mean((y_train - A_output)**2)
c:\Users\Srikanta\Documents\Intro to Machine Learning\programming_assignment_1\models\neural_networks.py:460: RuntimeWarning: overflow encountered in square
  val_metric = np.mean((y_val - val_output)**2)
c:\Users\Srikanta\Documents\Intro to Machine Learning\programming_assignment_1\models\neural_networks.py:428: RuntimeWarning: invalid value encountered in multiply
  error_hidden_2 = np.dot(error_output, self.W_output.T) * (1 - np.power(A2, 2))  # Derivative of tanh is (1 - tanh^2)


Tested params: {'lr': 0.001, 'epochs': 11000, 'n_hidden_2': 51}, Score: nan
Testing params: {'lr': 1e-07, 'epochs': 7000, 'n_hidden_2': 51}
Skipping params: {'lr': 1e-07, 'epochs': 7000, 'n_hidden_2': 51} due to high score: 52599.20835970355
Tested params: {'lr': 1e-07, 'epochs': 7000, 'n_hidden_2': 51}, Score: 52599.20835970355
Testing params: {'lr': 1e-05, 'epochs': 13000, 'n_hidden_2': 6}
Skipping params: {'lr': 1e-05, 'epochs': 13000, 'n_hidden_2': 6} due to high score: 20650.487323065456
Tested params: {'lr': 1e-05, 'epochs': 13000, 'n_hidden_2': 6}, Score: 20650.487323065456
Testing params: {'lr': 0.001, 'epochs': 19000, 'n_hidden_2': 66}


c:\Users\Srikanta\Documents\Intro to Machine Learning\programming_assignment_1\.venv\Lib\site-packages\numpy\core\_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
c:\Users\Srikanta\Documents\Intro to Machine Learning\programming_assignment_1\models\neural_networks.py:459: RuntimeWarning: overflow encountered in square
  train_metric = np.mean((y_train - A_output)**2)
c:\Users\Srikanta\Documents\Intro to Machine Learning\programming_assignment_1\models\neural_networks.py:460: RuntimeWarning: overflow encountered in square
  val_metric = np.mean((y_val - val_output)**2)
c:\Users\Srikanta\Documents\Intro to Machine Learning\programming_assignment_1\models\neural_networks.py:428: RuntimeWarning: invalid value encountered in multiply
  error_hidden_2 = np.dot(error_output, self.W_output.T) * (1 - np.power(A2, 2))  # Derivative of tanh is (1 - tanh^2)


Tested params: {'lr': 0.001, 'epochs': 19000, 'n_hidden_2': 66}, Score: nan
Testing params: {'lr': 1e-07, 'epochs': 5000, 'n_hidden_2': 6}
Skipping params: {'lr': 1e-07, 'epochs': 5000, 'n_hidden_2': 6} due to high score: 65581.39574187237
Tested params: {'lr': 1e-07, 'epochs': 5000, 'n_hidden_2': 6}, Score: 65581.39574187237
Testing params: {'lr': 1e-07, 'epochs': 11000, 'n_hidden_2': 66}
Skipping params: {'lr': 1e-07, 'epochs': 11000, 'n_hidden_2': 66} due to high score: 43337.74593770218
Tested params: {'lr': 1e-07, 'epochs': 11000, 'n_hidden_2': 66}, Score: 43337.74593770218
Testing params: {'lr': 0.001, 'epochs': 5000, 'n_hidden_2': 6}
Skipping params: {'lr': 0.001, 'epochs': 5000, 'n_hidden_2': 6} due to high score: 52358.02782280216
Tested params: {'lr': 0.001, 'epochs': 5000, 'n_hidden_2': 6}, Score: 52358.02782280216
Testing params: {'lr': 0.001, 'epochs': 5000, 'n_hidden_2': 36}


c:\Users\Srikanta\Documents\Intro to Machine Learning\programming_assignment_1\.venv\Lib\site-packages\numpy\core\_methods.py:118: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
c:\Users\Srikanta\Documents\Intro to Machine Learning\programming_assignment_1\models\neural_networks.py:459: RuntimeWarning: overflow encountered in square
  train_metric = np.mean((y_train - A_output)**2)
c:\Users\Srikanta\Documents\Intro to Machine Learning\programming_assignment_1\models\neural_networks.py:460: RuntimeWarning: overflow encountered in square
  val_metric = np.mean((y_val - val_output)**2)
c:\Users\Srikanta\Documents\Intro to Machine Learning\programming_assignment_1\models\neural_networks.py:428: RuntimeWarning: invalid value encountered in multiply
  error_hidden_2 = np.dot(error_output, self.W_output.T) * (1 - np.power(A2, 2))  # Derivative of tanh is (1 - tanh^2)


Tested params: {'lr': 0.001, 'epochs': 5000, 'n_hidden_2': 36}, Score: nan
Testing params: {'lr': 0.0001, 'epochs': 11000, 'n_hidden_2': 6}
Skipping params: {'lr': 0.0001, 'epochs': 11000, 'n_hidden_2': 6} due to high score: 26404.83361538519
Tested params: {'lr': 0.0001, 'epochs': 11000, 'n_hidden_2': 6}, Score: 26404.83361538519
Testing params: {'lr': 1e-07, 'epochs': 5000, 'n_hidden_2': 6}
Skipping params: {'lr': 1e-07, 'epochs': 5000, 'n_hidden_2': 6} due to high score: 65467.43810670397
Tested params: {'lr': 1e-07, 'epochs': 5000, 'n_hidden_2': 6}, Score: 65467.43810670397
Testing params: {'lr': 0.0001, 'epochs': 17000, 'n_hidden_2': 96}
Skipping params: {'lr': 0.0001, 'epochs': 17000, 'n_hidden_2': 96} due to high score: 33498.56462901738
Tested params: {'lr': 0.0001, 'epochs': 17000, 'n_hidden_2': 96}, Score: 26584.275132495837
Testing params: {'lr': 1e-07, 'epochs': 7000, 'n_hidden_2': 81}
Skipping params: {'lr': 1e-07, 'epochs': 7000, 'n_hidden_2': 81} due to high score: 49317

## Model Performance ##

In [24]:
linear_scores = []
ffn_scores = []
combined_scores = []

for i, (train_set, test_set) in enumerate(cross_validator.cross_validation(data_train, n_splits=2, n_repeats=5, random_state=42, stratify=False)):

    train_data = train_set.to_numpy()
    X_train = train_data[:,:-1]
    y_train = train_data[:,-1:]

    test_data = test_set.to_numpy()
    X_test = test_data[:,:-1]
    y_test = test_data[:,-1:]

    # Check if the shapes of X_train and X_val are not equal
    if X_train.shape[1] != X_test.shape[1]:
        # print(f"Shape mismatch between training and validation sets, skipping params: {params}")
        continue
    
    linear = LinearNetwork(config)
    _, linear_val_losses = linear.linear_regression(X_train,y_train,X_test,y_test,epochs=best_params_linear['epochs'],lr=best_params_linear['lr'],patience=np.inf)

    ffn = FeedForwardNetwork(config,n_input=X_train.shape[1],n_hidden_1=best_params_ffn['n_hidden'],n_hidden_2=best_params_ffn['n_hidden'],n_output=y_train.shape[1])
    _, ffn_val_losses, _ = ffn.train(X_train,y_train,X_test,y_test,epochs=best_params_ffn['epochs'],lr=best_params_ffn['lr'],patience=np.inf)

    autoE = AutoEncoder(config,n_input=X_train.shape[1],n_encoder=best_params_auto['n_encoder'])
    losses = autoE.train(X_train, max_epochs=best_params_auto['epochs'], lr=best_params_auto['lr'])
    combined = CombinedModel(autoE,n_hidden_2=best_params_combined['n_hidden_2'],n_output=y_test.shape[1])
    _, combined_val_losses, _ = combined.train(X_train,y_train,X_test,y_test,epochs=best_params_combined['epochs'], lr=best_params_combined['lr'],patience=np.inf)


    linear_score = np.min(linear_val_losses)
    ffn_score = np.min(ffn_val_losses)
    combined_score = np.min(combined_val_losses)
    
    linear_scores.append(linear_score)
    ffn_scores.append(ffn_score)
    combined_scores.append(combined_score)

avg_score_linear = np.mean(linear_scores)
avg_score_ffn = np.mean(ffn_scores)
avg_score_combined = np.mean(combined_scores)

print(f"Linear Model Tested params: {best_params_linear}, Average Score: {avg_score_linear}")
print(f"FFN Model Tested params: {best_params_ffn}, Average Score: {avg_score_ffn}")
print(f"Combined Model Tested params: {best_params_combined}, Average Score: {avg_score_combined}")

print(f"Linear Model Scores: {linear_scores}")
print(f"FFN Model Scores: {ffn_scores}")
print(f"Combined Model Scores: {combined_scores}")



Linear Model Tested params: {'lr': 0.01, 'epochs': 20000}, Average Score: 5437.229857771791
FFN Model Tested params: {'lr': 0.0001, 'epochs': 8000, 'n_hidden': 72}, Average Score: 6413.998034747997
Combined Model Tested params: {'lr': 1e-05, 'epochs': 19000, 'n_hidden_2': 51}, Average Score: 5704.171624577029
Linear Model Scores: [7393.463525634592, 3361.6506758423907, 3269.280937946132, 7281.165725630679, 7844.847515288771, 3497.5946821491066, 7268.175881200254, 2995.0895737393257, 2599.632841191785, 8861.397219094873]
FFN Model Scores: [9526.646182737022, 1692.78491962975, 2151.4437232320215, 8347.852019165213, 8382.195966333851, 6812.029753503698, 15280.45925892775, 2957.0385163995807, 915.8536736890944, 8073.676333861985]
Combined Model Scores: [8416.180353810572, 1576.9937208560202, 1804.5479187539943, 10764.231172258087, 8700.837890321956, 1521.5472876242993, 12053.436992100273, 2232.825391797534, 1394.4851877361737, 8576.630330511383]


In [25]:
from scipy import stats
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Combine all scores into a single array, and create an array of labels
scores = np.concatenate([linear_scores, ffn_scores, combined_scores])
labels = ['Linear'] * len(linear_scores) + ['FFN'] * len(ffn_scores) + ['Combined'] * len(combined_scores)

# Conduct ANOVA
anova_result = stats.f_oneway(linear_scores, ffn_scores, combined_scores)
print(f"ANOVA result: F-statistic = {anova_result.statistic}, p-value = {anova_result.pvalue}")

# If ANOVA shows significant differences, conduct post-hoc testing with Tukey's HSD
if anova_result.pvalue < 0.05:
    tukey = pairwise_tukeyhsd(endog=scores, groups=labels, alpha=0.05)
    print(tukey)


ANOVA result: F-statistic = 0.16898436718024284, p-value = 0.8454083687364071


In [26]:
from scipy import stats

t_stat, p_val = stats.ttest_ind(linear_scores, ffn_scores)
print(f"Linear Model vs. FFN Model: t-statistic = {t_stat}, p-value = {p_val}")

# Comparing Linear Model vs. Combined Model
t_stat, p_val = stats.ttest_ind(linear_scores, combined_scores)
print(f"Linear Model vs. Combined Model: t-statistic = {t_stat}, p-value = {p_val}")

# Comparing FFN Model vs. Combined Model
t_stat, p_val = stats.ttest_ind(ffn_scores, combined_scores)
print(f"FFN Model vs. Combined Model: t-statistic = {t_stat}, p-value = {p_val}")

Linear Model vs. FFN Model: t-statistic = -0.6027395468351565, p-value = 0.55420065904851
Linear Model vs. Combined Model: t-statistic = -0.168548444692268, p-value = 0.8680313204668435
FFN Model vs. Combined Model: t-statistic = 0.35875517002439733, p-value = 0.7239517104896738


## Archive ##

In [27]:
# data = data_train.to_numpy()
# X_train = data[:,:-1]
# y_train = data[:,-1:]

In [28]:
# data_test = data_val.to_numpy()
# X_val = data_test[:,:-1]
# y_val = data_test[:,-1:]

In [29]:
# autoE = AutoEncoder(config,n_input=X_train.shape[1],n_encoder=4)

# autoE.train(X_train, max_epochs=30000, lr=0.0001)

In [30]:
# combined = CombinedModel(autoE,n_hidden_2=100,n_output=1)

# MSEs, val_metrics, final_mse = combined.train(X_train,y_train,X_val,y_val,epochs=10000,lr=0.000001)

In [31]:
# import matplotlib.pyplot as plt

# plt.plot(MSEs)
# plt.plot(val_metrics)

In [32]:
# ffn = FeedForwardNetwork(config,n_input=X_train.shape[1],n_hidden_1=100,n_hidden_2=100,n_output=1)

# MSEs, val_metrics, final_mse = ffn.train(X_train,y_train,X_val,y_val,5000,0.000001)

In [33]:
# import matplotlib.pyplot as plt

# plt.plot(MSEs)
# plt.plot(val_metrics)

In [34]:
# linear = LinearNetwork(config)

# losses = linear.linear_regression(X_train,y_train,X_val,y_val)
# losses